<a href="https://colab.research.google.com/github/mahe62/AS_proto/blob/main/analisis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# pip install -U scikit-learn

In [63]:
import pandas as pd
import re
import numpy as np
import pickle
import joblib

from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

In [64]:
df = pd.read_csv("./omicronlabel.csv")
df.head(5)

,Column1,stop_remove,label
0,0,casi 300 pasaporte mascarilla covid19 covid 1...,netral
1,1,lakukan menyikapi konfirmasi omicron indonesi...,netral
2,2,birylu sih gitu omicron desa ga nyediain ting...,negative
3,3,divisi kena omicron gw doang alhamdulillah neg...,negative
4,4,soukor 27k cases semalam bcs of omicron so mus...,negative


In [65]:
#konversi polaritas
def convert(polarity):
  if polarity == 'positive':
    return 1
  elif polarity == 'netral':
    return 0
  else:
    return -1

In [66]:
df['Polarity'] = df['label'].apply(convert)

In [67]:
X = df['stop_remove']
y = df['Polarity']

In [68]:
bow_transformer = CountVectorizer()
print(df['stop_remove'].shape)
X = bow_transformer.fit_transform(df['stop_remove'])

print(X.toarray())
print('Shape of Sparse Matrix: ', X.shape)
print('Amount of Non-Zero occurences: ', X.nnz)

#menyimpan CountVectorizer
filename1 = './CountVectorizer1.pkl'
pickle.dump(bow_transformer, open(filename1, 'wb'))

#TFID Transform
tf_transform = TfidfTransformer(use_idf=False).fit(X)
X = tf_transform.transform(X)
print(X.shape)

#menyimpan TFID
filename1 = './TfidfTransformer1.pkl'
pickle.dump(tf_transform, open(filename1, 'wb'))

(279,)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Shape of Sparse Matrix:  (279, 1680)
Amount of Non-Zero occurences:  3323
(279, 1680)


In [69]:
density = (100.0 * X.nnz / (X.shape[0] * X.shape[1]))
print ('Density: {}'.format((density)))

Density: 0.7089520395972009


In [70]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2)

In [71]:
nb = MultinomialNB()
nb.fit(X_train, y_train)
pred = nb.predict(X_test)

In [72]:
print(classification_report(y_test,pred))

              precision    recall  f1-score   support

          -1       0.48      0.70      0.57        23
           0       0.00      0.00      0.00        13
           1       0.48      0.55      0.51        20

    accuracy                           0.48        56
   macro avg       0.32      0.42      0.36        56
weighted avg       0.37      0.48      0.42        56



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [73]:
from io import StringIO
classification = classification_report(y_test,pred)
s= StringIO(classification)
with open('classification.csv','w')as f:
  for line in s: 
    f.write(line)

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [74]:
print(accuracy_score(y_test,pred))

0.48214285714285715


In [75]:
accuracy = accuracy_score(y_test,pred)
a = np.asarray([accuracy])
np.savetxt("accuracy.csv",a, delimiter=",", fmt='%s')

In [76]:
df.columns

Index(['Column1', 'stop_remove', 'label', 'Polarity'], dtype='object')

In [77]:
Fix = df[['stop_remove', 'label', 'Polarity']]

In [78]:
Fix.to_csv('./hasil_analisis.csv')

In [79]:
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [80]:
filename = 'model_analisis.pkl'
pickle.dump(model, open(filename, 'wb'))

In [81]:
loaded_model = pickle.load(open(filename,'rb'))
result = loaded_model.score(X_test,y_test)
print(result)

0.4642857142857143


In [82]:
loaded_model.predict(X_test)

array([ 1, -1,  1, -1,  1, -1, -1, -1, -1, -1,  1,  1, -1, -1, -1,  1, -1,
       -1,  1,  1, -1,  1,  1, -1, -1,  1, -1, -1,  1, -1, -1,  1,  1,  1,
       -1, -1, -1,  1, -1,  1, -1, -1, -1, -1, -1, -1, -1,  1, -1,  1, -1,
       -1, -1, -1, -1,  1])